In [1]:
# Загрузка и обработка данных
import pandas as pd

# Визуализация данных
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import warnings

pd.options.display.max_columns = None
warnings.filterwarnings("ignore")

In [2]:
dates_columns = ['date', 'createdDate', 'changedDate']
data = pd.read_csv('../data/prepared/outcomes.csv', parse_dates=dates_columns)

In [4]:
data.tail()

,date,categoryName,payee,comment,createdDate,changedDate,outcomeAccountName,outcome,outcomeCurrencyShortTitle
1187,2024-03-20,Проезд,Самолет,Билеты домой и из дома,2024-03-20 12:39:37,2024-03-20 09:39:52,Тинькофф,21020.0,RUB
1188,2024-03-20,Кафе и рестораны,Столовая ОЦРВ,NaN,2024-03-20 12:41:05,2024-03-20 09:41:14,Тинькофф,460.0,RUB
1189,2024-03-20,Кафе и рестораны,Ресторан Вега,NaN,2024-03-21 12:53:07,2024-03-21 09:53:20,Тинькофф,265.0,RUB
1190,2024-03-20,Проезд,Яндекс такси,NaN,2024-03-21 12:53:17,2024-03-21 09:53:37,Тинькофф,212.0,RUB
1191,2024-03-21,Кафе и рестораны,Столовая ОЦРВ,NaN,2024-03-21 12:53:35,2024-03-21 09:53:46,Тинькофф,460.0,RUB


## Посмотрим на выбросы в данных

## Рассчитаем базовые статистики

In [5]:
mean = data['outcome'].mean()
median = data['outcome'].median()
percentile_20 = data['outcome'].quantile(q=0.2)
percentile_40 = data['outcome'].quantile(q=0.4)
percentile_60 = data['outcome'].quantile(q=0.6)
percentile_80 = data['outcome'].quantile(q=0.8)
maximun = data['outcome'].max()
minimun = data['outcome'].min()
standard_deviation = data['outcome'].std()

result = f"""
<< Базовая статистика >>

+ Средняя сумма трат: {mean} руб

+ Медианная сумма трат: {median} руб
+ 20% моих трат меньше, чем {percentile_20} рублей
+ 40% моих трат меньше, чем {percentile_40} рублей
+ 60% моих трат меньше, чем {percentile_60} рублей
+ 80% моих трат меньше, чем {percentile_80} рублей

+ Максималная трата: {maximun} руб
+ Минимальная трата: {minimun} руб

* Стандартное отклонение: {standard_deviation} руб
"""
print(result)


<< Базовая статистика >>

+ Средняя сумма трат: 522.3953104026846 руб

+ Медианная сумма трат: 259.99 руб
+ 20% моих трат меньше, чем 110.60000000000001 рублей
+ 40% моих трат меньше, чем 200.0 рублей
+ 60% моих трат меньше, чем 323.252 рублей
+ 80% моих трат меньше, чем 533.0 рублей

+ Максималная трата: 80000.0 руб
+ Минимальная трата: 10.0 руб

* Стандартное отклонение: 2465.0404729779907 руб



Теперь мне интересно посмотреть на эти басовые статистики не в целом, а по дням

In [40]:
data_by_days = data.groupby('date', as_index=False) \
                  .agg({'outcome' : 'sum'})
all_dates = pd.date_range(start=data_by_days['date'].min(), end=data_by_days['date'].max(), freq='D').to_frame(index=False, name='date')

data_by_days = pd.merge(all_dates, data_by_days, on='date', how='left').fillna({'outcome': 0})

mean = data_by_days['outcome'].mean()
median = data_by_days['outcome'].median()
percentile_20 = data_by_days['outcome'].quantile(q=0.2)
percentile_40 = data_by_days['outcome'].quantile(q=0.4)
percentile_60 = data_by_days['outcome'].quantile(q=0.6)
percentile_80 = data_by_days['outcome'].quantile(q=0.8)
maximun = data_by_days['outcome'].max()
minimun = data_by_days['outcome'].min()
standard_deviation = data_by_days['outcome'].std()

result = f"""
<< Базовая статистика за день>>

+ Средняя сумма трат (в день): {mean} руб

+ Медианная сумма (в день) трат: {median} руб
+ 20% моих трат (в день) меньше, чем {percentile_20} рублей
+ 40% моих трат (в день) меньше, чем {percentile_40} рублей
+ 60% моих трат (в день) меньше, чем {percentile_60} рублей
+ 80% моих трат (в день) меньше, чем {percentile_80} рублей

+ Максималная трата за день: {maximun} руб
+ Минимальная трата за день: {minimun} руб

* Стандартное отклонение: {standard_deviation} руб
"""
print(result)


<< Базовая статистика за день>>

+ Средняя сумма трат (в день): 1462.7903234501346 руб

+ Медианная сумма (в день) трат: 927.51 руб
+ 20% моих трат (в день) меньше, чем 363.7 рублей
+ 40% моих трат (в день) меньше, чем 778.97 рублей
+ 60% моих трат (в день) меньше, чем 1194.98 рублей
+ 80% моих трат (в день) меньше, чем 2009.4 рублей

+ Максималная трата за день: 22039.0 руб
+ Минимальная трата за день: 0.0 руб

* Стандартное отклонение: 2030.589365446051 руб



Тут все уже не так радужно. Медианно, я трачу около 1000 рублей в день, что довольно много. Всего 20% дней я тратил меньше, чем 364 рубля.

In [37]:
px.box(data, y='outcome')

Приятно видеть, что расходы в 5-6 тысяч рублей распознаются как выбросы. Видимо я, все-таки довольно экономный

In [38]:
px.box(data_by_days, y='outcome')

Мы видим 1 очень выделяющийся выброс, связанный с покупкой курса по аналитике за 80 тыс. рублей. Я думаю, что его стоит удалить, чтобы он не смещал дальнейшие графики

In [39]:
data = data[data['outcome'] < 80000]

## Посмотрим на динамику трат с течением времени

In [41]:
px.line(data_by_days, x='date', y='outcome')

В среднем, сумма трат довольно стабильная, но есть редкие выбросы, связанные с крупными покупками

#### Рассмотрим тренд моих трат

In [45]:
plot_data = data.groupby('date', as_index=False) \
                .agg({'outcome' : 'sum'}) \
                .sort_values('date')

plot_data['rolling'] = plot_data['outcome'].rolling(10, center=True).mean()

# Используем px.scatter для добавления линии тренда к исходным данным дохода
fig = px.scatter(plot_data, x='date', y='outcome', trendline="ols",
                 title='Динамика моих доходов с течением времени (Скользящее среднее)')

# Добавляем линию скользящего среднего отдельно
fig.add_scatter(x=plot_data['date'], y=plot_data['rolling'], mode='lines', name='Скользящее среднее')

fig.show()

Итого, мы можем увидеть, что мои все-таки возрасли с марта 2023 года. 